In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"

s3_bucket = "s3://mist-secorapp-{env}/sticky-client/sticky-client-{env}/".format(env=env)

date_day = "2020-03-0*"
hr = '*'

s3_path = "dt={day}/hr={hr}/*.seq".format(env=env, day=date_day, hr=hr)

s3_path = s3_bucket + s3_path
print(s3_path)




s3://mist-secorapp-production/sticky-client/sticky-client-production/dt=2020-03-0*/hr=*/*.seq


In [2]:
rdd = spark.sparkContext.sequenceFile(s3_path)


In [3]:
rdd.first()

(2868370,
 bytearray(b'{"Version":3,"WC":"00-56-cd-1d-96-91","Sticky":false,"When":"2020-03-01T00:00:09.641772209Z","Assoc":{"OrgID":"4a7f417d-a4b1-4452-804e-589a29dff85d","SiteID":"28a11fdb-7d32-4abc-b8c0-c1eda38b9f92","BSSID":"5c-5b-35-9c-16-83","WLAN":"fe3e1e10-c7fa-43b2-9a87-486174a05f1b","Band":"2.4","AP":"5c-5b-35-9f-6d-00","SSID":"WPA Prod","AvgRSSI":-67}}'))

In [14]:
df_sticky = rdd.map(lambda x: json.loads(x[1])).toDF()
df_sticky.show(5)

/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+--------------------+------+-------+-----------------+--------------------+
|               Assoc|Sticky|Version|               WC|                When|
+--------------------+------+-------+-----------------+--------------------+
|[OrgID -> 4a7f417...| false|      3|00-56-cd-1d-96-91|2020-03-01T00:00:...|
|[OrgID -> 2010156...|  true|      3|0c-dd-24-af-1b-8a|2020-03-01T00:00:...|
|[OrgID -> d4d2e00...|  true|      3|88-da-1a-79-b6-be|2020-03-01T00:00:...|
|[OrgID -> ee61f48...| false|      3|54-33-cb-e5-fa-9f|2020-03-01T00:00:...|
|[OrgID -> d4d2e00...| false|      3|88-da-1a-79-b6-be|2020-03-01T00:00:...|
+--------------------+------+-------+-----------------+--------------------+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode

df_sticky_flat = df_sticky.filter(col("Sticky")==True).select("WC", "When", "Assoc.OrgID", "Assoc.SiteID")
df_sticky_flat.show(5)

+-----------------+--------------------+--------------------+--------------------+
|               WC|                When|               OrgID|              SiteID|
+-----------------+--------------------+--------------------+--------------------+
|0c-dd-24-af-1b-8a|2020-03-01T00:00:...|20101562-d4f8-41a...|a77c1b1a-38d5-492...|
|88-da-1a-79-b6-be|2020-03-01T00:00:...|d4d2e000-beca-467...|9821201c-4244-426...|
|00-19-70-98-87-2e|2020-03-01T00:00:...|2a28c8e5-b65f-44a...|fea217ca-a63f-421...|
|40-74-e0-0e-8c-ce|2020-03-01T00:01:...|20101562-d4f8-41a...|499c118b-4b95-446...|
|e4-02-9b-d4-85-be|2020-03-01T00:03:...|20101562-d4f8-41a...|d0e76aa0-e140-456...|
+-----------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# df_sticky.filter(col("Sticky")==True).sel

In [18]:
df_sticky_flat.groupBy("OrgID",'SiteID').count().show()

Py4JJavaError: An error occurred while calling o333.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13091 in stage 12.0 failed 1 times, most recent failure: Lost task 13091.0 in stage 12.0 (TID 13103, localhost, executor driver): org.apache.hadoop.fs.s3a.AWSS3IOException: getFileStatus on s3://mist-secorapp-production/sticky-client/sticky-client-production/dt=2020-03-09/hr=11/1_10_00000000000002780515.seq: com.amazonaws.services.s3.model.AmazonS3Exception: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: A71CF6ACFAC7EF13), S3 Extended Request ID: B+QJHDsH1g9ijKQoQIvRBllb0IRRVYNl8V0YB3sW2k0op0sThh4YwnRJ8BnZ473yV6KVoOd00ZM=: Bad Request (Service: Amazon S3; Status Code: 400; Error Code: 400 Bad Request; Request ID: A71CF6ACFAC7EF13)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:178)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:101)
	at org.apache.hadoop.fs.s